# Constraint-based sampling for metabolic flux analysis (MFA)

Here we jump into the (for now) final step in our metabolic flux analysis (MFA) pipeline. After constructing an INCA script, running it in MATLAB and reimporting the data we're now here. This example notebook will guide you through different ways to integrate your MFA results into COBRA models and how to make them more reliable.

In [ ]:
import pandas as pd
import numpy as np
import time
import ast
import sys
import escher
import cobra
from gurobipy import Model as GRBModel
import re
from BFAIR.INCA import INCA_reimport
from BFAIR.INCA.sampling import *

#### INCA re-import

First, let's reimport the data using our `BFAIR INCA_reimport` tools

The data is taken from a different folder

In [2]:
%cd /Users/matmat/Documents/Random data/Stefano
filename = 'data/MFA_modelInputsData/TestFile.mat'

/Users/matmat/Documents/Random data/Stefano


Then we move back to our example folder to get the remaining input

In [3]:
%cd /Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples
#filename = 'data/MFA_modelInputsData/TestFile.mat'
simulation_info = pd.read_csv('data/MFA_modelInputsData/Re-import/experimentalMS_data_I.csv')
simulation_id = 'WTEColi_113C80_U13C20_01'

/Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples


Here we re-import the INCA output

In [4]:
reimport_data = INCA_reimport()
(fittedData,
 fittedFluxes,
 fittedFragments,
 fittedMeasuredFluxes,
 fittedMeasuredFragments,
 fittedMeasuredFluxResiduals,
 fittedMeasuredFragmentResiduals,
 simulationParameters) = reimport_data.reimport(
    filename,
    simulation_info,
    simulation_id
)

Here we import the model

In [5]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [6]:
original_solution = model.optimize()
original_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,-0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,-8.673617e-18
RNTR2c2,0.026541,-8.673617e-18


Let's see what happens now when we add our new bound constraints

In [7]:
model = add_constraints(model, fittedFluxes)
model.optimize()

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpbfykpmtw.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpfssynv7h.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


<Solution infeasible at 0x7f90f16ebd30>

Oh... That's not good... Well that sucks, seems like we have to deal with an infeasible solution. There are two straight forward ways: exclusion and relaxation

#### Dealing with infeasible solutions - exclusion

The easier way to deal with this issue is to simply exclude the constraints that render a model infeasible. We can do that by adding the calculated bounds one by one. If we come across a reaction whose bounds cause trouble, we restart the process and skip this one. This might have to be done a few times to exclude all troublemakers. the `add_feasible_constraints()` functions takes care of that for us. Let's reset the model first.

In [8]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [9]:
model, problems = add_feasible_constraints(model, fittedFluxes)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp5fcln55c.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpz3fr5nuy.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpprj_75h2.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmptlaq31dt.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding RPI
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpnj3q8x47.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmplq9volku

The `model` is our newly constrained model and the problematic reactions can be listed in `problems`.

In [10]:
model

Name,iJO1366
Memory address,0x07f90d7085100
Number of metabolites,1805
Number of reactions,2583
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [11]:
problems

['ASPTA', 'RPI']

Now let's see what an effect these new bounds had on the predicted growth rate (the objective value) of our model

In [12]:
new_bounds_solution = model.optimize()
new_bounds_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-1.104597
EX_co2_e,14.276815,0.000000
EX_cobalt2_e,-0.000007,-0.000000
DM_4crsol_c,0.000062,0.000000
...,...,...
RNDR4,0.000000,0.000000
RNDR4b,0.000000,0.000000
RNTR1c2,0.000000,0.000000
RNTR2c2,0.007461,0.000000


And here's the star of the show, our sampling method. We trust our models because... we have to! And because smart people that knew what they were doing set them up. So in order to gain more confidence in our MFA data, we sample the model after adding the calculated bound for some of the reactions and re-calculate the fluxes a number of time. Then, we take the mean and take that as the most trustworthy calculated flux. These fluxes can be visualized, for example in tools like `Escher`

In [13]:
sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp_hzd_wv4.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpqxa6s6nm.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [14]:
sampled_fluxes

,EX_cm_e,EX_cmp_e,EX_co2_e,EX_cobalt2_e,DM_4crsol_c,DM_5drib_c,DM_aacald_c,DM_amob_c,DM_mththf_c,EX_colipa_e,...,UREAtex,RNDR2b,UREAtpp,RNDR3,RNDR3b,RNDR4,RNDR4b,RNTR1c2,RNTR2c2,RNTR3c2
0,0.0,0.0,33.130059,-4.226792e-12,3.926493e-11,0.009919,0.0,3.519145e-13,1.441202e-06,3.693090e-08,...,-1.483480e-07,3.204476e-07,-1.483480e-07,1.235241e-08,5.663156e-10,3.272136e-07,5.579514e-08,3.524056e-07,0.024964,7.306110e-09
1,0.0,0.0,33.130053,-1.392340e-11,1.293714e-10,0.009919,0.0,1.160044e-12,3.894675e-09,5.040690e-08,...,-9.798014e-08,3.195293e-07,-9.798014e-08,2.329130e-08,4.275718e-13,3.200205e-07,5.579310e-08,3.513558e-07,0.024964,7.280100e-09
2,0.0,0.0,33.130037,-1.274466e-11,1.184141e-10,0.009919,0.0,1.061773e-12,3.601880e-06,3.844763e-08,...,-4.268446e-09,4.636872e-07,-4.268446e-09,2.192144e-08,4.241527e-13,3.187825e-07,5.579290e-08,2.202373e-07,0.024966,7.392176e-09
3,0.0,0.0,33.128476,-1.821008e-10,1.680811e-09,0.009918,0.0,1.507431e-11,2.299773e-06,4.333025e-08,...,-4.851415e-06,4.534183e-07,-4.851415e-06,2.027542e-07,2.413880e-10,4.842569e-07,5.400506e-08,3.515948e-07,0.024963,6.953523e-09
4,0.0,0.0,33.128356,-1.885852e-10,1.741062e-09,0.009918,0.0,1.561467e-11,2.115541e-06,4.332798e-08,...,-1.212795e-06,1.694460e-06,-1.212795e-06,2.059411e-07,3.973657e-09,4.008973e-08,5.451746e-08,3.523691e-07,0.024963,9.244430e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,29.696591,-3.864019e-07,3.541661e-06,0.009682,0.0,3.161101e-08,6.873980e-02,3.600470e-04,...,-4.105861e-02,5.645206e-02,-4.105861e-02,1.725498e-03,1.052910e-04,1.845468e-02,1.191174e-02,4.375347e-02,0.104932,2.434310e-04
96,0.0,0.0,29.658254,-4.087625e-07,3.751404e-06,0.009679,0.0,3.349216e-08,7.330653e-02,3.597408e-04,...,-4.912884e-02,8.511124e-03,-4.912884e-02,1.728159e-03,1.412328e-04,1.845674e-02,1.190217e-02,4.372254e-02,0.105520,2.276813e-04
97,0.0,0.0,29.743620,-4.412597e-07,4.054348e-06,0.009137,0.0,3.620900e-08,2.541690e-02,5.786948e-05,...,-4.913649e-02,8.582313e-03,-4.913649e-02,1.725053e-03,1.389311e-04,1.893347e-02,1.195197e-02,4.279476e-02,0.105682,6.177158e-04
98,0.0,0.0,29.768632,-3.999644e-07,3.667768e-06,0.009263,0.0,3.274207e-08,1.225141e-02,4.139917e-04,...,-4.106454e-02,8.512561e-03,-4.106454e-02,1.724712e-03,1.388716e-04,1.885891e-02,1.190058e-02,4.511300e-02,0.053641,5.711134e-04


In [15]:
fluxes_sampling = reshape_fluxes_escher(sampled_fluxes)

In [16]:
sampled_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = fluxes_sampling).display_in_notebook()
sampled_flux_map

There are also other ways to cosolidate the MFA calculations and the constraint based flux predictions. One of these is lMOMA (linear Minimization Of Metabolic Adjustment). MOMA assumes that the fluxes before and after adding the new constraints should be similar, so it aims to predicting an optimum for the newly constrined model while keeping the differences to the original model small. We suggest using pFBA (parsimonious Flux Balance Analysis) instead of regular FBA for this step as pFBA aims to keep the overal fluxes low.

In [17]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model, problems = add_feasible_constraints(model, fittedFluxes)
pfba_solution = cobra.flux_analysis.pfba(model)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp4v8792n3.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp0qoe3pwt.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpxknckv9p.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpk5k2t0pj.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding RPI
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpstjwb0t7.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp3jah29nh.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
---------------------------
Total number of restarts: 2
add_feasible_constraints takes 0h: 0min: 37sec to run
--- end ---


In [18]:
moma_after_MFA = cobra.flux_analysis.moma(model=model, solution=pfba_solution, linear=True)

For some reason I get weird values when I just look at the objective value here, both for pFBA and for lMOMA, so I just look at the biomass function instead. This should produce a similar result as the regular FBA prediction after adding the new constraints

In [19]:
moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.27614926420028335

In [20]:
print(f'Before adding new constraints:             {original_solution.objective_value:.2f}')
print(f'After adding new constraints:              {new_bounds_solution.objective_value:.2f}')
print(f'After adding new constraints + pFBA, objV: {pfba_solution.objective_value:.2f}')
print(f'After adding new constraints + pFBA, BM:   {pfba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]:.2f}')
print(f'After adding new constraints + MOMA, objV: {moma_after_MFA.objective_value:.2f}')
print(f'After adding new constraints + MOMA, BM:   {moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]:.2f}')

Before adding new constraints:             0.98
After adding new constraints:              0.28
After adding new constraints + pFBA, objV: 370.91
After adding new constraints + pFBA, BM:   0.28
After adding new constraints + MOMA, objV: 0.00
After adding new constraints + MOMA, BM:   0.28


Of course also the MOMA results can be visualized with `Escher`. The `reshape_fluxes_escher()` can take both pandas DataFrames or cobra solutions as an input.

In [21]:
fluxes_moma = reshape_fluxes_escher(moma_after_MFA)

In [22]:
moma_flux_map = escher.Builder('e_coli_core.Core metabolism',
                               reaction_data = fluxes_moma).display_in_notebook()
moma_flux_map

#### Dealing with infeasible solutions - relaxation

Another way of dealing with infeasible models is to relax the added constraints to the point that it works again. You will need to have the Gurobi solver installed for this. The same principle is used in the `BFAIR thermo` tools. For that we add our constraints to a model that will now be infeasible.

In [23]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [24]:
model = add_constraints(model, fittedFluxes)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp3of2_ak3.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpu7z9w1na.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


In [25]:
model.optimize()

<Solution infeasible at 0x7f90dcf62af0>

Then we make use of the handy `bound_relaxation()` function that will test our model to figure out which of these added bounds need to be adjusted and return a DataFrame that describes the affected functions and the gravity of the suggested changes. If we allow this function to be `desctructive` it will adjust the input model right away.

In [26]:
bound_relaxation(model, fittedFluxes, destructive=True)

--- start ---
bound_relaxation takes 0h: 0min: 0sec to run
--- end ---


,bound_change,subsystem
reaction,,
ASPTA,-1.3,Alanine and Aspartate Metabolism
DAPDC,-0.2,Threonine and Lysine Metabolism


Let's see if our model is feasible now.

In [27]:
relaxed_solution = model.optimize()
relaxed_solution

,fluxes,reduced_costs
EX_cm_e,0.000000e+00,0.0
EX_cmp_e,0.000000e+00,0.0
EX_co2_e,1.125075e+01,0.0
EX_cobalt2_e,-3.213772e-12,-0.0
DM_4crsol_c,7.849232e-09,0.0
...,...,...
RNDR4,0.000000e+00,0.0
RNDR4b,0.000000e+00,0.0
RNTR1c2,0.000000e+00,0.0
RNTR2c2,0.000000e+00,0.0


In [28]:
relaxed_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

1.285508972317679e-07

And now we're also at the point where we can let the power of constraint based models work for us and make use of the tools mentioned above in order to adjust the fluxes calculated using MFA so that they nicely fit into our model.

In [29]:
relaxed_sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpgwa6w0c1.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp8zv8ebre.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [30]:
relaxed_fluxes_sampling = reshape_fluxes_escher(relaxed_sampled_fluxes)

In [31]:
relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = relaxed_fluxes_sampling).display_in_notebook()
relaxed_flux_map